**Import Libraries**

In [2]:
#!pip install plotly
#import plotly.express as px
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.decomposition import PCA
import requests
from bs4 import BeautifulSoup
import re
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import seaborn as sns
from scipy import stats
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.utils import shuffle
from warnings import simplefilter
simplefilter('ignore', category=FutureWarning)

Please note, we will be collaborating via Git. Please find our project at https://github.com/ppich1169/fertilityProjectCs109

# Milestone 1: Proposal

Over the past fifty years, fertility rates in the US have plummeted and are currently at a historic low. Conversations about why fertility has fallen so substantially and how we can address the implications of this shift for government programs like social security have been quite salient in recent public discourse and in the 2024 election cycle. 

Interestingly, there is significant variation in fertility rates across US states. We’d like to understand the relative importance of various factors in determining a state’s fertility rate. 

We plan to run a multiple regression of fertility rate (can get state-by-state here from the CDC’s National Center for Health Statistics) on a number of regressors

**Goal:** Create a regression that can predict the fertility rate of a state. Then, analyze coefficients and/or use causal inference to understand why fertility rate is going down

# Milestone 2: Preprocessing

## 1. Access the data that you will be using for the final project by downloading, collecting, or scraping* from the relevant source(s) and 2. Load the data into a Jupyter notebook and understand the data by examining, among other characteristics of interest, data missingness, imbalance, and scaling issues.

**TL;DR** Hi! We didn't know how long to make our doc but wanted to get as much feedback as possible so included everything here. We are more than happy to procide an abridged version. Essentially, we aquired data with a combination of webscraping and downloading from websites. Most of the data has some element of missingess as there are some gaps in years. We would fill this missingness by creating a linear regression on the data we have, and filling in our unknown data. We also needed to scale some of our data into the range 0-1. Finally, we proprocessed that data by making some categorical variables ordinal, others one-hot encoded, and finally putting all our individal data sets together (linking on year and state). Thank you for taking the time to read this!!

### Response Variable

Our response variable is **fertility rate by state over time** which can be found at https://www.cdc.gov/nchs/pressroom/sosmap/fertility_rate/fertility_rates.htm. 

We accessed it via download and saved it as `fertility_rate_census.csv`. 

Please note, fertility rate is defined as  **total number of births per 1,000 women aged 15-44** and our dataset looks at fertility rate for each of the 50 states over 9 years (2014-2022).

In [127]:
fertility_rate = pd.read_csv('data/fertility_rate_census.csv')
print(fertility_rate.isna().sum(axis=0))
fertility_rate.describe()

YEAR              0
STATE             0
FERTILITY RATE    0
BIRTHS            0
URL               0
dtype: int64


,YEAR,FERTILITY RATE,BIRTHS
count,451.000000,451.000000,451.000000
mean,2018.008869,60.057871,75783.962306
std,2.588850,6.420758,86837.134690
min,2014.000000,44.300000,5133.000000
25%,2016.000000,56.000000,21758.500000
50%,2018.000000,60.200000,55971.000000
75%,2020.000000,63.500000,86486.000000
max,2022.000000,80.000000,502879.000000


In [128]:
pivoted_fertility = fertility_rate.pivot(index='STATE', columns='YEAR', values='FERTILITY RATE')
pivoted_fertility[pivoted_fertility.isna().any(axis=1)]

YEAR,2014,2015,2016,2017,2018,2019,2020,2021,2022
STATE,,,,,,,,,
District of Columbia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.3


Things to Consider:

**Missingness**: While there is no empty cell in the original dataset, we see if we pivot it by year, Washington DC only shows up in 2022. We can simply delete this row as Washington DC isn't technically a state. This is Missing at Random because we know the reason there was no Washington DC from 2014-2021 is that DC isn't considered a state

**Imbalance**: Since we aren't looking at different classes there is no imbalance

**Scaling**: By considering Fertility Rate (and not Number of Births), we are essentially normalizing our data as we are dividing it by total population. This will be enough in order to scale the data as it takes into account the populations of each state such that no state is overly weighed. In addition, since fertility rate is a response variable, not a predictor, we don't have to consider how large our data is in relation to other variables. Thus, we don't have to scale it any further.

**Other**: We need to encode the state variable as categorical if it will be used in modeling by creating dummy variables or one-hot encoding. This issue will be true for all datasets!

### Predictors
_We used our previous knowledge and assumptions to create an **X** dataset of predictors that we believe may influence fertility rates_

Because fertility rate is evaluated statewide , and states vary significantly in population size, we have decided that for all of the predictors, we are going to essentially **normalize** them by looking at the percentage of each state that fall into a specific category. 

## CENSUS DATA

We began by collecting essential demographic data from the U.S. Census Bureau's website (https://www.census.gov) to analyze trends across the U.S. population. This data included indicators such as socioeconomic status, foreign-born populations, education levels, and racial demographics. We organized the data into separate data frames—`SPM_df`, `foreignborn_df`, `education_dfs`, and `race_dfs`—to facilitate efficient management and analysis.

The **SPM (Supplemental Poverty Measure)** data, loaded from `SPM.csv`, provides a broad measure of poverty across states, capturing household income levels below the poverty line and accounting for both cash income and non-cash benefits. Although this data originates from the Census Bureau, we accessed it through Statista (https://www.statista.com/statistics/312701/percentage-of-population-foreign-born-in-the-us-by-state/), where it was already aggregated and prepared for analysis. Upon examination, we found no missing values, making it straightforward to integrate into our analysis without additional preprocessing. The structure, with a single column for each state’s poverty estimate, is simple and well-suited for analysis.

The **foreign-born population dataset**, sourced directly from Census data in `foreignborn2022.csv`, records the percentage of foreign-born individuals by state for 2022. This dataset offers insights into immigration trends, which may impact fertility rates, and was found to be free from missing values. The data is cleanly structured with only two columns (`State` and `Percent`), making it ready for immediate use in analysis.

The **education data** files, loaded from various `*education.csv` files, provide shares of educational attainment across different age groups, offering a nuanced view of educational levels within each state. Each file includes detailed demographic breakdowns with hierarchical labels representing educational categories, such as `Total`, `Percent`, and specific demographic groupings (like age, gender). When analyzing these files, we found a complex column structure and hierarchical labels, which will require additional preprocessing for effective analysis. Moreover, each state had some missing values across specific columns, likely due to unavailable data for certain demographic subgroups. These missing values will need to be handled through imputation or selective removal of columns with excessive gaps.

Finally, the **race data** files, loaded from various `*race.csv` files, provide demographic breakdowns by race for each state. This dataset mirrors the structure of the education data, with hierarchical labels that represent various racial categories and demographic details. Similar to the education data, the race data was found to have missing values and a complex structure. The intricate labeling and demographic specificity will require careful preprocessing to flatten the hierarchical labels and manage missing values, ensuring consistency with other datasets.

### SPM Dataset

**Missingness**: The SPM data, sourced from `SPM.csv`, contains no missing values, making it easy to integrate into the analysis without any imputation. The completeness of this dataset ensures we can use it directly to assess the relationship between poverty levels and fertility rates across states.

**Imbalance**: This dataset represents one measurement (poverty estimate) per state, so there are no categorical classes that could lead to imbalance. Each state has a single poverty estimate, meaning no particular state is over- or underrepresented in terms of poverty measurement.

**Scaling**: The SPM dataset is already normalized, as it provides the percentage of households below the poverty line for each state. This percentage-based format allows us to make direct comparisons between states without additional scaling, as the data is inherently comparable across geographic areas.

### Foreign-Born Population Dataset

**Missingness**: The foreign-born population dataset, from `foreignborn2022.csv`, is also complete with no missing values. This allows for straightforward inclusion in the analysis, with no need for handling missing data.

**Imbalance**: This dataset has no categorical classes and represents a single percentage (foreign-born population) per state. There is, therefore, no issue of imbalance in this dataset, as each state is equally represented with one percentage value.

**Scaling**:  The data is already in percentage form, representing the share of foreign-born individuals in each state's population. This normalized format allows for direct comparison across states without further scaling, as each percentage reflects a relative measure rather than an absolute count.

### Education Dataset

**Missingness**: The education data files, loaded from `*education.csv`, contain missing values across some columns, particularly in specific demographic breakdowns within each state. This missingness appears sporadically, likely due to data limitations in certain subgroups or categories. Handling these missing values will require careful consideration, with options to either impute values based on related data points or exclude columns with excessive gaps.

**Imbalance**:  
Imbalance is not a significant concern for the education dataset, as each state is represented with comprehensive data on various age and educational attainment levels. The dataset does not rely on categorical classes that could lead to imbalance issues, and all states provide data across similar categories of educational attainment.

**Scaling**:  
Since the education data is provided by age group, we need to scale it by the population size of each age group within each state to ensure accuracy when comparing educational attainment percentages. Scaling by age group population will allow us to make accurate cross-state comparisons of educational attainment levels, as it controls for differences in the age distributions across states. This normalization will make the educational attainment percentages more representative and comparable in relation to each state's age demographics.

### Race Dataset

**Missingness**:  
The race data files, loaded from `*race.csv`, exhibit some missing values, likely due to unavailable data for certain racial categories or demographic subgroups within states. Similar to the education dataset, we will need to address these missing values through imputation or by selectively excluding columns with significant gaps.

**Imbalance**:  
This dataset contains detailed racial breakdowns across states, with some racial categories potentially having fewer entries than others. If analyzing racial subgroups individually, imbalance may occur due to underrepresentation of smaller demographic groups in certain states, which could lead to biased interpretations. Grouping data by broader racial categories or considering population weights may help to mitigate this issue.

**Scaling**:  
The data is already in percentage form, representing the share of individuals of differet races in each state's population. 

**Load and explore data**

In [129]:
# Load SPM data and display structure
SPM_df = pd.read_csv('data/SPM.csv')
print("SPM Data Structure:")
print(SPM_df.head())
print("\nMissing Values in SPM Data:")
print(SPM_df.isna().sum())

# Load foreign-born data and display structure
foreignborn_df = pd.read_csv('data/foreignborn.csv', header=None)
foreignborn_df.columns = foreignborn_df.iloc[0]  # Set the header to the first row
foreignborn_df = foreignborn_df.drop(0)  # Drop the first row as it is now the header
foreignborn_df.reset_index(drop=True, inplace=True)  # Reset the index if needed
print("\nForeign Born Data Structure:")
print(foreignborn_df.head())
print("\nMissing Values in Foreign Born Data:")
print(foreignborn_df.isna().sum())
foreignborn_df = foreignborn_df.rename(columns={"2022.0": "2022", "2010.0": "2010"})

# Function to load files matching a pattern
def load_data_files(file_pattern, description):
    files = glob.glob(file_pattern)
    data_frames = {}
    
    for file in files:
        try:
            # Try loading with comma separator
            df = pd.read_csv(file, sep=',', on_bad_lines='skip')
            data_frames[file] = df
            print(f"\n{description} Data from {file}:")
            print(df.head())
            print("\nMissing Values in this DataFrame:")
            print(df.isna().sum(), "\n")
        except pd.errors.ParserError:
            print(f"Parser error in {file}. Trying with a different delimiter or settings...")
            try:
                # Try loading with a tab
                df = pd.read_csv(file, sep='\t', on_bad_lines='skip')
                data_frames[file] = df
                print(f"\n{description} Data from {file} (loaded with tab separator):")
                print(df.head())
                print("\nMissing Values in this DataFrame:")
                print(df.isna().sum(), "\n")
            except Exception as e:
                print(f"Could not load {file} even with alternate parsing: {e}")
        except Exception as e:
            print(f"Unexpected error loading {file}: {e}")
    
    return data_frames

# Load education data 
education_dfs = load_data_files("data/*education.csv", "Education")

# Load race data
race_dfs = load_data_files("data/*race.csv", "Race")


SPM Data Structure:
   Year      States Estimate
0  2022     Alabama     16.3
1  2022      Alaska     10.8
2  2022     Arizona     12.3
3  2022    Arkansas     16.6
4  2022  California     11.7

Missing Values in SPM Data:
Year        0
States      0
Estimate    0
dtype: int64

Foreign Born Data Structure:
0       State  2022.0  2010.0
0  California    26.7    27.2
1  New Jersey    23.5    21.0
2    New York    22.7    22.2
3     Florida    21.7    19.4
4      Nevada    18.9    18.8

Missing Values in Foreign Born Data:
0
State     0
2022.0    0
2010.0    0
dtype: int64

Education Data from data/2015education.csv:
                                  Label (Grouping) Alabama!!Total!!Estimate  \
0                    AGE BY EDUCATIONAL ATTAINMENT                      NaN   
1                        Population 18 to 24 years                  468,319   
2                   Less than high school graduate                   70,992   
3      High school graduate (includes equivalency)            

We also care about religion, political makeup, and whether certain abortion laws are in place (all of which are not in the census). We will find them different ways.

## RELIGION DATA

We decided to determine people's **religiousness** based off of a state's adherence rate (number of people who adhere to their religion across 1000 ) which can be found at (https://www.thearda.com/us-religion/maps/us-state-maps)

We accessed it by webscraping and saved it in `religion_data.csv` in our data folder

In [130]:
url2020 = "https://www.thearda.com/us-religion/maps/us-state-maps"
url2010 = "https://www.thearda.com/us-religion/maps/us-state-maps?color=orange&m1=2_2_9999_2010"
url2000 = "https://www.thearda.com/us-religion/maps/us-state-maps?color=orange&m1=2_2_9999_2000"
response = requests.get(url2000)
soup = BeautifulSoup(response.content, 'html.parser')

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
    
script_tag = soup.find('script', text=re.compile(r'usa_map_div299992000_data = '))
script_content = script_tag.string
start_index = script_content.find('usa_map_div299992000_data =')
semicolon_index = script_content.find(';', start_index)
mapData = script_content[start_index:semicolon_index]

quoted_strings = re.findall(r'"(.*?)"', mapData)
values_strings = re.findall(r'(\d+\.\d+)', mapData)
year = []
for i in range(len(values_strings)):
    year.append(2000)

last_two_chars = [s[-2:] for s in quoted_strings]
print(last_two_chars)
print(len(values_strings))

file_paths = [
    'data/AllReligionAdherence_2000.csv',
    'data/AllReligionAdherence_2010.csv',
    'data/AllReligionAdherence_2020.csv'
]

dfs = []

for file_path in file_paths:
    file_name = os.path.basename(file_path)
    year = file_name.split('_')[-1].split('.')[0]
    df = pd.read_csv(file_path)
    df['Year'] = year
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

['UT', 'ND', 'DC', 'SD', 'MA', 'RI', 'MN', 'OK', 'WI', 'NY', 'NE', 'LA', 'IA', 'NM', 'PA', 'CT', 'NJ', 'AR', 'TX', 'IL', 'AL', 'MS', 'KY', 'MO', 'TN', 'KS', 'ID', 'NH', 'SC', 'WY', 'CA', 'NC', 'OH', 'GA', 'MT', 'MD', 'IN', 'MI', 'VA', 'FL', 'DE', 'AZ', 'CO', 'VT', 'ME', 'HI', 'WV', 'AK', 'NV', 'WA', 'OR']
51


/var/folders/kg/wxm8v9d91mv575kb5wsgxf4h0000gp/T/ipykernel_28808/2974073243.py:12: DeprecationWarning:

The 'text' argument to find()-type methods is deprecated. Use 'string' instead.



In [131]:
religion_data = pd.read_csv('data/religion_data.csv')
print(religion_data.isna().sum(axis=0))
religion_data.describe()

State                      0
Adherence Rate per 1000    0
Year                       0
dtype: int64


,Adherence Rate per 1000,Year
count,153.000000,153.00000
mean,489.381569,2010.00000
std,101.640693,8.19178
min,272.190000,2000.00000
25%,414.860000,2000.00000
50%,492.770000,2010.00000
75%,555.040000,2020.00000
max,791.060000,2020.00000


Things to Consider:

**Missingness**: After webscraping ARDA, there are no missing values in any cells, but definitely in the number of years (153 vs 451 above). This is because this index is only calculated every 10 years (2000, 2010, 2020). We have to decide how we want to impute this data. One option is to create a line going through the 2000, 2010, and 2020 datapoints, and then impute the data that would sit on this line for 2011-2019 and 2021-2022. 

**Imbalance**: Although the actual number of state datapoints are equal and fine, the source of the imbalance stems from which religions do these data sets stem from. All the main categories listed on the Association of Religious Digital Archives are branches of Christianity. I have no idea if these are actually the most popular religions in the US or is the source biased towards it.

**Scaling**: Since adherence rates range widely (e.g., from 272.19 to 791.06), standardizing or normalizing values might be beneficial, particularly if this data will be used for machine learning or statistical modeling. Standardization (e.g., Z-score) could be applied if you want each adherence rate centered around zero, while min-max normalization scales values between a range like [0, 1].

## POLITICS DATA

We decided to determine people's political orienation based off of the Cook Partisan Voting Index (Cook PVI), which is a measure of each state's political leaning relative to the nation as a whole.

The primary challenge with using this index is that the methodology was switched in 2022 to weigh the last presidential election 0.75 and the second to last 0.25, as opposed to the even (50/50) weighting of both years that was used in prior years. We will either figure out how to reweight the outcomes based on the raw data if we can get it or exclude 2022 from our analysis.

The calculation of the index is described in more detail here: https://www.cookpolitical.com/cook-pvi

In [132]:
pol_df = pd.read_csv('data/cook_pvi_2022.csv')
print(pol_df.isna().sum(axis=0))
pol_df.describe()

State           0
2022_PVI        0
2020_Biden      0
2020_Trump      0
2016_Clinton    0
2016_Trump      0
dtype: int64


,State,2022_PVI,2020_Biden,2020_Trump,2016_Clinton,2016_Trump
count,51,51,51,51,51,51
unique,51,30,50,48,48,48
top,Alabama,R+11,49.50%,48.80%,27.50%,45.50%
freq,1,3,2,2,2,2


**Missingness and imbalance**: None in the dataset 

**Scaling:** We will follow a similar methodology to that used in "State-level Political Partisanship Strongly Correlates with Health Outcomes for US Children," (full citation below).* We converted the PVI's to numerical values, with negative values representing Democratic PVIs and positive numbers representing Republican ones (an arbitrary choice). Then, we scaled those values with sklearn's MinMaxScaler() so that states have a rating between 0 and 1 representing how conservative they are, with 1 being most conservatve and 0 being most liberal.

We accessed the data for 2022 from this source: https://datawrapper.dwcdn.net/0djXs/2/ and saved it in cook_pvi_2022.csv in our data folder. We tried scraping the website but the data is not in the html but instead pulled from a source so we were unable to access the data using the same strategy as HW 1.

The 2022 data is saved in `data/cook_pvi_2022.csv.`

*Paul, M., Zhang, R., Liu, B. et al. State-level political partisanship strongly correlates with health outcomes for US children. Eur J Pediatr 181, 273–280 (2022). https://doi.org/10.1007/s00431-021-04203-y

## ABORTION DATA

We decided to determine state's **abortion laws** based off of how late into pregnancy, abortion is legally allowed which can be found at https://lawatlas.org/datasets/abortion-bans. We chose this dataset because it is the only one on the internet showing abortion bans in the 2014-2022 time frame and how they change (most just show abortion bans now)

We accessed it by downloading it, converting from xlsx to csv, and saved it in `abortion_data.csv` in our data folder

The main thing to consider is that **just because abortion is legal, doesn't mean it is accessible**. Many states may technically allow abortion but only have one clinic, so its not attainable. That said, we have chosen this metric (when is abortion legal), to coincide with current political debate about whether abortion should be legalized. 

There will be significant preprocessing required as the data is in format `Effective Date`, `Valid Through Date` for each law which must simply be converted into year (whichever law was the majority of the year), and each ban `6 weeks`, `8 weeks` etc is categorical! It would make more sense to simply make a variable listing the latest week aborition is legal (0,6,8,12,52 etc).  

In [133]:
abortion_data = pd.read_csv('data/abortion_data.csv')

In [134]:
columns = ["State","Effective Date","Valid Through Date","Bans_gest_4 weeks postfertilization (6 weeks LMP) " ,"Bans_gest_6 weeks postfertilization (8 weeks LMP) " ,"Bans_gest_8 weeks postfertilization (10 weeks LMP)","Bans_gest_10 weeks postfertilization (12 weeks LMP) " ,"Bans_gest_12 weeks postfertilization (14 weeks LMP)","Bans_gest_13 weeks postfertilization (15 weeks LMP)","Bans_gest_16 weeks postfertilization (18 weeks LMP) ","Bans_gest_18 weeks postfertilization (20 weeks LMP)","Bans_gest_19 weeks postfertilization (21 weeks LMP)","Bans_gest20 weeks postfertilization (22 weeks LMP)","Bans_gest_21 weeks postfertilization (23 weeks LMP) " ,"Bans_gest_22 weeks postfertilization (24 weeks LMP)","Bans_gest_24 weeks postfertilization (26 weeks LMP)","Bans_gestViability","Bans_gest_Fetus is capable of feeling pain","Bans_gest_3rd trimester"]
abortion_data = abortion_data[columns]
new_names = {
    "State": "state",
    "Effective Date": "start",
    "Valid Through Date": "end",
    "Bans_gest_4 weeks postfertilization (6 weeks LMP) ": "4",
    "Bans_gest_6 weeks postfertilization (8 weeks LMP) ": "6",
    "Bans_gest_8 weeks postfertilization (10 weeks LMP)": "8",
    "Bans_gest_10 weeks postfertilization (12 weeks LMP) ": "10",
    "Bans_gest_12 weeks postfertilization (14 weeks LMP)": "12",
    "Bans_gest_13 weeks postfertilization (15 weeks LMP)": "13",
    "Bans_gest_16 weeks postfertilization (18 weeks LMP) ": "16",
    "Bans_gest_18 weeks postfertilization (20 weeks LMP)": "18",
    "Bans_gest_19 weeks postfertilization (21 weeks LMP)": "19",
    "Bans_gest20 weeks postfertilization (22 weeks LMP)": "20",
    "Bans_gest_21 weeks postfertilization (23 weeks LMP) ": "21",
    "Bans_gest_22 weeks postfertilization (24 weeks LMP)": "22",
    "Bans_gest_24 weeks postfertilization (26 weeks LMP)": "24",
    "Bans_gestViability": "24", #chose via google
    "Bans_gest_Fetus is capable of feeling pain": "25", #chose via google
    "Bans_gest_3rd trimester": "28" #chose via google
}
abortion_data = abortion_data.rename(columns=new_names)

abortion_processed = abortion_data[['state', 'start', 'end']].copy()
abortion_processed['latest_abortion'] = abortion_data[abortion_data.columns].apply(
    lambda row: next((int(col) for col in abortion_data.columns  if str(row[col]) == "1"), 40), axis=1)

abortion_processed['start'] = pd.to_datetime(abortion_processed['start'])
abortion_processed['end'] = pd.to_datetime(abortion_processed['end'])
abortion_processed['year'] = abortion_processed['start'].dt.year
abortion_processed['length'] = (abortion_processed['end'] - abortion_processed['start']).dt.days
abortion_processed = abortion_processed.loc[abortion_processed.groupby(['state', 'year'])['length'].idxmax()]
abortion_processed = abortion_processed.drop(columns=['start', 'end', 'length'])

abortion_processed.head()

,state,latest_abortion,year
0,Alabama,20,2018
2,Alabama,20,2019
3,Alabama,20,2022
4,Alaska,40,2018
5,Arizona,18,2018


In [135]:
print(abortion_processed.isna().sum(axis=0))
print("missing from entire dataframe", 50*5-len(abortion_processed))
abortion_processed.describe()

state              0
latest_abortion    0
year               0
dtype: int64
missing from entire dataframe 113


,latest_abortion,year
count,137.000000,137.000000
mean,25.598540,2019.729927
std,10.328758,1.647206
min,4.000000,2018.000000
25%,20.000000,2018.000000
50%,20.000000,2019.000000
75%,40.000000,2021.000000
max,40.000000,2022.000000


Things to Consider:

**Missingness**: We see while there are no null cells in our dataset, there are 113 "missing cells" from what would be expected (one for every state for every year). This is simply because of how our dataset was made -- there is only a new row if a new law is enacted (not one per year). Thus, if we are missing a row for a certain year, we can simply **impute** the data by copying the previous row. 

**Imbalance**: There is no imbalance as there is no population sampling. 

**Scaling**: We don't need to scale the data by itself but might scale data as a whole later so they don't have stds that are too high. 

**Other**: We decided to change our data from a 1-hot encoded dataset, to 1 quantitative varaible. We did this because there seems to be a clear relatonship between the previously encoded categories (a 10 week ban is stronger than a 15 week ban and less strong than a 4 week ban). Thus, we made it ordinal. In additon, note that if there was no abortion ban, we arbitrarily set this ordinal variable to 40, the length of the average pregancy. It might make some sense to add an **indicator variable** to be 0 if there is no abortion ban at all (40 weeks) and 1 if there is an abortion ban. 

Some issues we preliminary have considered before even inspecting the data includes:


- **under reporting immigration status**: via google, people tend to underreport whether they are immigrants. This is potentially a missingness issue

-  **multicollinearity**: There is most likely a relationship between racial background / household income and an individual's birth country (immigration status) so we can't use both as predictors in the same equation. We don't know how strong this correlation will be so are not worried yet, but would love to talk about it with a TF

- **is this just too much data??**: looking at each of these attributes for each state for each year may just be too many dimensions. Is there really a big difference accross years? Should we just look at 1? If so, which? 

Now we are going to import and inspect each dataset, looking for missingness, imbalance, and scaling issues!

## 3. Understand and describe the preprocessing required such that data is in a form amenable to later downstream tasks such as visualizing and modeling, as is appropriate to the specific project goals.



One of our biggest considerations is that our predictor variables, X, is essentially 3d (reshaped), not 2d. We are looking at a bunch of predictors **over state** and **over time**. This might make our predictions complicated, especially because we are trying to see **why** fertility is changing over time (so the reason that there are less babies in 2022 can't simply be that it is 2022). Thus, it might make more sense to choose only **one year** to regress on. Also, it may be hard to **visualize** multiple years and states simultaniously as, for example, if we drew a map and colored each state by predictor category, we would have to choose one specific time to do so (and vice versa).  PLEASE LET US KNOW WHAT YOU THINK!

It also may make sense to just try to minimize predictors in general as we don't want to get too specific!

### This is how we envision generally preprocessing...

As you can see, we generally started preprocessign above. For example, we changed abortion laws from categorical variables into an orderinal variable. In addition, as seen below, we standardized the poltical data by scaling it between 0 and 1. 

Below is the preprocessing for political data. We scale the Cook PVI scores to be on a scale of 0 to 1, where 1 is most conservative and 0 is least conservative. We drop DC because it is not a state. There is no missingness in this dataset.

In [136]:
#Drop DC because it's not a state: 
pol_df = pol_df[pol_df['State'] != 'District of Columbia']
pol_df.reset_index(drop=True, inplace=True)

unscaled_ratings=[]
for i in range(len(pol_df)):
    magnitude = re.search(r"(?<=\+).*", pol_df['2022_PVI'][i])
    magnitude = int(magnitude.group())
    
    if pol_df['2022_PVI'][i][0] == 'R':
        unscaled_ratings.append(magnitude)
    else:
        unscaled_ratings.append(magnitude * -1)

pol_df['unscaled_rating'] = unscaled_ratings

pol_rating_scaler = MinMaxScaler()
scaled_pol_ratings = pol_rating_scaler.fit_transform(pol_df['unscaled_rating'].values.reshape(-1, 1))

pol_df['scaled_rating'] = scaled_pol_ratings
pol_df.head()

pol_df['unscaled_rating'].min()

np.int64(-16)

To prepare the education dataset for analysis, we began by iterating through each *education.csv file. We extracted the year from each file name to ensure we could track data across different time periods. Within each file, we removed the rows labeled "Population 25 years and over" along with the following nine rows, as they did not provide useful information for our analysis. We then isolated columns relevant to our study, retaining only the primary label column and columns ending with "!!Percent Females!!Estimate," which represent the estimated educational attainment percentages for females in each state.

For each age group (e.g., "Population 18 to 24 years"), we calculated weighted values for educational attainment percentages based on the total population for that group. This scaling ensures that educational attainment percentages are accurately reflected relative to each age group’s size, enabling meaningful comparisons across states. After processing each file, we created a consolidated data frame containing columns for Label, Year, State, Female Estimate (weighted educational attainment percentage), and Population. Finally, we dropped rows where "Female Estimate" values were missing, resulting in a cleaned and structured education_df ready for analysis.

For the race dataset, we followed a similar iterative approach, processing each *race.csv file individually and extracting the year from each file name. Each file was loaded while skipping the first two lines to focus only on relevant data, and we added a Year column to track temporal changes. To simplify the dataset, we removed columns representing less relevant racial categories (such as “American Indian or Alaska Native,” “Native Hawaiian or Pacific Islander”) and redundant summary columns (like “Total” and “Footnotes”). We also excluded rows containing data for "United States" as an aggregate, ensuring that only state-level data remained. Finally, we dropped any rows where "White" values were missing, creating a clean and consistent structure across files. The processed race data frames were combined into a single race_df for use in further analysis.

These steps produced two organized, cleaned data frames, education_df and race_df, each containing essential demographic details by year and state. The education data is now scaled appropriately by age group population, and both datasets are free of unnecessary or missing values, allowing for more accurate analysis of the relationships between education, race, and fertility rates across states.

In [137]:
df_list = []

# Process each education file
for file in education_dfs:
    # Get year from the file name
    year = file.split('/')[-1][:4]

    # Load CSV file
    df = pd.read_csv(file)

    # Drop "Population 25 years and over" and the nine rows following it
    pop_25_index = df[df.iloc[:, 0] == "Population 25 years and over"].index
    if not pop_25_index.empty:
        df = df.drop(index=range(pop_25_index[0], pop_25_index[0] + 10)).reset_index(drop=True)

    # Keep only the first column (label) and columns ending with "!!Percent Females!!Estimate"
    label_column = df.columns[0]
    columns_to_keep = [label_column] + [col for col in df.columns if col.endswith("!!Percent Females!!Estimate")]
    df = df[columns_to_keep]

    # Process each age group section to calculate weighted averages
    selected_data = []
    current_population = None
    total_population = 0 
    for index, row in df.iterrows():
        label = row[label_column]

        # Check if row is a population group 
        if "Population" in label:
            # Get population number and set as title
            try:
                total_population = int(label.split()[1]) if label.split()[1].isdigit() else None
            except ValueError:
                total_population = None
            current_population = total_population 
        elif current_population and "Percent" not in label:
            # Calculate weighted value for educational attainment percentages based on the total population
            for col in df.columns[1:]:  # Exclude the label column
                state = col.split("!!")[0]
                percent_str = row[col]
                try:
                    percent = float(str(percent_str).replace("%", "").strip()) if percent_str else None
                except ValueError:
                    percent = None  # Set as None if conversion fails

                if percent is not None:
                    scaled_value = (percent / 100) * total_population
                else:
                    scaled_value = None

                # Append the result
                selected_data.append({
                    "Label": label,
                    "Year": year,
                    "State": state,
                    "Female Estimate": scaled_value,
                    "Population": total_population
                })

    #Append to df_list
    temp_df = pd.DataFrame(selected_data)
    df_list.append(temp_df)

# Combine all data framesZASxzdw
education_df = pd.concat(df_list, ignore_index=True)

# Drop rows NaN
education_df.dropna(subset=["Female Estimate"], inplace=True)


In [138]:
df_list = []

# Process each race file
for file_path in race_dfs:
    # Get the year
    year = file_path.split('/')[-1].split('race')[0]
    
    try:
        # Load the CSV
        df = pd.read_csv(file_path, skiprows=2, delimiter=',')
        
        # Add the year
        df['Year'] = year
        
        # Drop columns
        columns_to_drop = ['American Indian or Alaska Native', 'Native Hawaiian or Pacific Islander', 'Total', 'Footnotes']
        df = df.loc[:, ~df.columns.str.contains('|'.join(columns_to_drop), na=False)]
        
        # Remove rows with "United States" 
        df = df[~df.iloc[:, 0].str.strip().str.lower().eq("united states")]
        
        # Remove rows where "White" is NaN
        df = df.dropna(subset=["White"])
        
        # Append
        df_list.append(df)
    
    except pd.errors.ParserError:
        print(f"Could not parse {file_path}. Skipping this file.")
    except Exception as e:
        print(f"An error occurred with file {file_path}: {e}")

# Concatenate
race_df = pd.concat(df_list, ignore_index=True)

We will also **impute/delete missingness** as described in each section above. This is especially relevent when we are missing certain years.
Finally, we will combine all our data into one DF and **make the state variable categorical.**

# Starting Milestone 3/4: Prelim Modeling

## First off, we impute all ourn misising data!

We started off by imputing/cleaning the data as described above. Our goal is to have 450 rows per data frame (50 states, 9 years from 2014-2022) and them to combine them along state and year axes. 

## Isabella Preprocessing -- to 450

### Impute: 

In [148]:
# List of years and states
years = list(range(2014, 2023))
states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 
    'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 
    'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 
    'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 
    'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 
    'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 
    'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 
    'Wisconsin', 'Wyoming'
]

# Define demographic columns
demographic_columns = ["White", "Black", "Hispanic", "Asian", "Multiple Races"]

# Create a DF with combinations of years and states
all_combinations = pd.DataFrame([(year, state) for year in years for state in states], columns=['Year', 'Location'])

all_combinations['Year'] = all_combinations['Year'].astype(int)
race_df['Year'] = race_df['Year'].astype(int)


# Merge all_combinations
race_df_full = all_combinations.merge(race_df, on=['Year', 'Location'], how='left')

# Convert to numeric
for col in demographic_columns:
    race_df_full[col] = pd.to_numeric(race_df_full[col], errors='coerce')

# Impute function
def impute_multiple_columns(row, df, columns):
    for col in columns:
        if pd.notnull(row[col]):
            continue 

        prev_year_value = df[(df['Location'] == row['Location']) & (df['Year'] == row['Year'] - 1)][col].values
        next_year_value = df[(df['Location'] == row['Location']) & (df['Year'] == row['Year'] + 1)][col].values
        if prev_year_value.size > 0 and next_year_value.size > 0:
            row[col] = (prev_year_value[0] + next_year_value[0]) / 2
        elif prev_year_value.size > 0:
            row[col] = prev_year_value[0]
        elif next_year_value.size > 0:
            row[col] = next_year_value[0]
    return row

# Impute
race_df_full = race_df_full.apply(impute_multiple_columns, axis=1, df=race_df_full, columns=demographic_columns)

# Drop rows
race_df_full.dropna(subset=demographic_columns, inplace=True)

# Fix montana
missing_years = [2015, 2016, 2017, 2018]

if not race_df_full[(race_df_full['Year'] == 2014) & (race_df_full['Location'] == 'Montana')].empty and \
   not race_df_full[(race_df_full['Year'] == 2019) & (race_df_full['Location'] == 'Montana')].empty:

    montana_data_2014 = race_df_full[(race_df_full['Year'] == 2014) & (race_df_full['Location'] == 'Montana')].iloc[0]
    montana_data_2019 = race_df_full[(race_df_full['Year'] == 2019) & (race_df_full['Location'] == 'Montana')].iloc[0]

    montana_missing_rows = pd.DataFrame({'Year': missing_years, 'Location': 'Montana'})
    for col in demographic_columns:
        if col in montana_data_2014 and col in montana_data_2019:
            avg_value = (montana_data_2014[col] + montana_data_2019[col]) / 2
            montana_missing_rows[col] = avg_value
        else:
            montana_missing_rows[col] = np.nan  # Handle missing columns gracefully

    race_df_full = pd.concat([race_df_full, montana_missing_rows], ignore_index=True)

# Reset the index 
race_df_full.reset_index(drop=True, inplace=True)

race_df_full


,Year,Location,White,Black,Hispanic,Asian,Multiple Races
0,2014,Alabama,0.6640,0.263,0.039,0.012,0.0180
1,2014,Alaska,0.6200,0.030,0.066,0.061,0.0740
2,2014,Arizona,0.5640,0.039,0.305,0.031,0.0210
3,2014,Arkansas,0.7360,0.153,0.070,0.012,0.0200
4,2014,California,0.3830,0.054,0.388,0.139,0.0310
...,...,...,...,...,...,...,...
445,2022,Wyoming,0.8190,0.007,0.109,0.006,0.0430
446,2015,Montana,0.8655,0.008,0.035,0.009,0.0225
447,2016,Montana,0.8655,0.008,0.035,0.009,0.0225
448,2017,Montana,0.8655,0.008,0.035,0.009,0.0225


In [ ]:
# Transpore DF
education_df['combined_label'] = education_df['Label'] + "_" + education_df['Population'].astype(str)
education_df['year'] = education_df['Year'] 
education_df['state'] = education_df['State'] 
education_pivoted = education_df.pivot_table(
    index=["year", "state"],
    columns="combined_label",
    values="Female Estimate",
    aggfunc='first'  # Assumes each (year, state, combined_label) combination is unique
)
education_pivoted.columns = education_pivoted.columns
education_pivoted.reset_index(inplace=True)
education_pivoted

# Create a DataFrame with all year-state combinations
years = list(range(2014, 2023))
states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 
    'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 
    'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 
    'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 
    'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 
    'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 
    'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 
    'Wisconsin', 'Wyoming'
]

all_combinations = pd.DataFrame([(year, state) for year in years for state in states], columns=['year', 'state'])
education_complete = all_combinations.merge(education_pivoted, on=['year', 'state'], how='left')

# Identify columns to impute
columns_to_impute = [col for col in education_complete.columns if col not in ['year', 'state']]

# Define imputation function
def impute_column(df, column):
    for i, row in df.iterrows():
        if pd.isnull(row[column]):
            state, year = row['state'], row['year']
            
            # Get values
            prev_years = df[(df['state'] == state) & (df['year'] < year)][['year', column]].dropna().sort_values('year', ascending=False)
            next_years = df[(df['state'] == state) & (df['year'] > year)][['year', column]].dropna().sort_values('year', ascending=True)
            
            # Use closest available value
            prev_value = prev_years[column].values[0] if not prev_years.empty else None
            next_value = next_years[column].values[0] if not next_years.empty else None
            
            # Impute
            if prev_value is not None and next_value is not None:
                df.at[i, column] = (prev_value + next_value) / 2
            elif prev_value is not None:
                df.at[i, column] = prev_value
            elif next_value is not None:
                df.at[i, column] = next_value

# Apply imputation
for column in columns_to_impute:
    impute_column(education_complete, column)


# Reset index
education_complete.dropna(inplace=True)
education_complete.reset_index(drop=True, inplace=True)

education_complete



,year,state,Bachelor's degree or higher_18,Bachelor's degree or higher_25,Bachelor's degree or higher_35,Bachelor's degree or higher_45,Bachelor's degree or higher_65,High school graduate (includes equivalency)_18,High school graduate (includes equivalency)_65,High school graduate or higher_25,High school graduate or higher_35,High school graduate or higher_45,High school graduate or higher_65,Less than high school graduate_18,Less than high school graduate_65,Some college or associate's degree_18,Some college or associate's degree_65,Bachelor's degree or higher_65,High school graduate or higher_65
0,2014,Alabama,1.692,7.300,10.710,11.025,11.180,5.094,12.610,22.250,31.185,39.285,51.415,2.340,23.660,8.874,9.620,17.355,56.615
1,2014,Alaska,1.422,8.050,11.830,14.490,18.070,6.804,9.555,23.225,32.865,42.165,56.355,2.430,12.545,7.362,5.005,24.700,62.205
2,2014,Arizona,1.602,7.450,10.605,12.510,14.950,5.004,12.480,22.175,29.890,39.285,55.250,2.628,22.750,8.766,7.605,18.590,57.590
3,2014,Arkansas,1.584,6.875,9.800,10.305,10.010,4.968,12.415,22.500,31.115,39.510,51.675,2.070,23.335,9.396,9.620,15.470,57.135
4,2014,California,2.070,9.375,12.950,13.815,16.380,4.806,10.985,22.200,28.875,36.765,50.635,1.836,17.940,9.288,7.670,21.060,55.315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,2022,Virginia,2.700,11.475,16.450,16.875,16.965,4.968,9.035,23.375,32.340,41.085,54.080,1.674,16.965,8.658,6.175,26.260,59.800
446,2022,Washington,2.430,10.300,14.560,15.120,17.485,4.734,8.905,23.350,31.745,41.310,57.785,2.520,16.120,8.298,6.695,23.010,60.970
447,2022,West Virginia,1.962,6.900,9.835,9.225,9.685,5.346,12.545,22.775,31.885,40.590,52.195,1.962,22.490,8.730,9.295,13.585,56.940
448,2022,Wisconsin,2.430,10.075,14.035,13.095,13.715,5.058,7.995,23.625,32.830,42.480,57.915,1.710,18.070,8.784,6.695,20.735,61.295


In [ ]:
# Rename columns
SPM_df = SPM_df.rename(columns={
    "Year": "year",
    "States": "state",
    "Estimate": "SPM"
})

# Filter for states and years
SPM_df = SPM_df[(SPM_df['state'].isin(states)) & (SPM_df['year'].between(2014, 2022))]

# Convert to numeric
SPM_df['SPM'] = pd.to_numeric(SPM_df['SPM'], errors='coerce')

# Find missing rows
states_missing_2020 = set(SPM_df[SPM_df['year'] == 2020]['state'])
states_all = set(SPM_df['state'].unique())
missing_states_2020 = states_all - states_missing_2020

imputed_rows = []

for state in missing_states_2020:
    # Get the SPM values for 2019 and 2021
    spm_2019 = SPM_df[(SPM_df['state'] == state) & (SPM_df['year'] == 2019)]['SPM'].values
    spm_2021 = SPM_df[(SPM_df['state'] == state) & (SPM_df['year'] == 2021)]['SPM'].values
    
    # Calculate the average
    if spm_2019.size > 0 and spm_2021.size > 0:
        spm_2020 = (spm_2019[0] + spm_2021[0]) / 2
        imputed_rows.append({'year': 2020, 'state': state, 'SPM': spm_2020})

# Imputed rows
imputed_df = pd.DataFrame(imputed_rows)
SPM_df = pd.concat([SPM_df, imputed_df], ignore_index=True)

# Sort
SPM_df = SPM_df.sort_values(by=['state', 'year']).reset_index(drop=True)

# Display
SPM_df




,year,state,SPM
0,2014,Alabama,19.3
1,2015,Alabama,18.7
2,2016,Alabama,17.1
3,2017,Alabama,16.6
4,2018,Alabama,16.6
...,...,...,...
445,2018,Wyoming,11.0
446,2019,Wyoming,9.8
447,2020,Wyoming,10.5
448,2021,Wyoming,11.2


In [ ]:
# Rename columns 
foreignborn_df = foreignborn_df.rename(columns={ "State": "state"})
foreignborn_df.columns.values[1] = "2022"
foreignborn_df.columns.values[2] = "2010"

# reshape 
foreignborn_df_long = pd.melt(
    foreignborn_df, 
    id_vars=["state"],
    value_vars=["2022", "2010"],
    var_name="year",
    value_name="foreignborn"
)

# Convert year to integer
foreignborn_df_long['year'] = foreignborn_df_long['year'].astype(int)

# Create a DataFrame with all combinations of state and year
years_to_impute = list(range(2010, 2023))
states = foreignborn_df_long['state'].unique()
all_combinations = pd.DataFrame([(state, year) for state in states for year in years_to_impute], columns=['state', 'year'])

# Merge with the original data
foreignborn_complete = pd.merge(all_combinations, foreignborn_df_long, on=['state', 'year'], how='left')

# Function to fill in missing years
def linear_impute(df):
    start_value = df.loc[df['year'] == 2010, 'foreignborn'].values[0]
    end_value = df.loc[df['year'] == 2022, 'foreignborn'].values[0]
    
    # Calculate the incremental change for each year
    step = (end_value - start_value) / 12  # 12 intervals between 2010 and 2022
    
    # Fill in each year with the interpolated value
    for i, year in enumerate(range(2011, 2022), start=1):
        df.loc[df['year'] == year, 'foreignborn'] = start_value + step * i
    
    return df

# Apply imputation function 
foreignborn_complete = foreignborn_complete.groupby('state').apply(linear_impute)

# Filter years
foreignborn_filtered = foreignborn_complete[(foreignborn_complete['year'] >= 2014) & (foreignborn_complete['year'] <= 2022)]
foreignborn_filtered = foreignborn_filtered.drop_duplicates(subset=['state', 'year'])

# Reset index
foreignborn_filtered.reset_index(drop=True, inplace=True)

foreignborn_filtered

/var/folders/kg/wxm8v9d91mv575kb5wsgxf4h0000gp/T/ipykernel_28808/4017145530.py:47: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,state,year,foreignborn
0,Alabama,2014,3.600
1,Alabama,2015,3.625
2,Alabama,2016,3.650
3,Alabama,2017,3.675
4,Alabama,2018,3.700
...,...,...,...
445,Wyoming,2018,3.000
446,Wyoming,2019,3.025
447,Wyoming,2020,3.050
448,Wyoming,2021,3.075


### Clean:

## Peter Preprocessing -- to 450

### Impute: 

We only had 3 years for religion adherence: 2000, 2010, and 2020. In order to impute the missing 8 years, we created a linear regression from 2010 to 2020. From there we predicted the adherence rates from 2011 to 2022. We eventually removed all duplicates and dropped all the years we don't want (we only want 2014-2022).

In [4]:
religion_data = pd.read_csv('data/religion_data.csv')

state_abbrev_to_name = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}

data_2010 = religion_data[religion_data['Year'] == 2010]
data_2020 = religion_data[religion_data['Year'] == 2020]

common_states = set(data_2010['State']).intersection(set(data_2020['State']))
data_2010 = data_2010[data_2010['State'].isin(common_states)]
data_2020 = data_2020[data_2020['State'].isin(common_states)]

data_2010 = data_2010.sort_values(by='State').reset_index(drop=True)
data_2020 = data_2020.sort_values(by='State').reset_index(drop=True)

years = np.array([2010, 2020]).reshape(-1, 1)
adherence_rates_2010 = data_2010['Adherence Rate per 1000'].values
adherence_rates_2020 = data_2020['Adherence Rate per 1000'].values

imputed_data = []
for state in common_states:
    adherence_rates = np.array([adherence_rates_2010[data_2010['State'] == state].item(),
                                adherence_rates_2020[data_2020['State'] == state].item()]).reshape(-1, 1)
    
    model = LinearRegression()
    model.fit(years, adherence_rates)
    
    for year in range(2011, 2023):
        imputed_rate = model.predict(np.array([[year]]))[0][0]
        imputed_data.append({'Year': year, 'State': state, 'Adherence Rate per 1000': imputed_rate})

imputed_df = pd.DataFrame(imputed_data)

religion_state_year = pd.concat([religion_data, imputed_df], ignore_index=True)
religion_state_year = religion_state_year.sort_values(by=['State', 'Year']).reset_index(drop=True)
religion_state_year = religion_state_year.drop_duplicates(subset=['State', 'Year'])
religion_state_year = religion_state_year[religion_state_year['State'] != 'DC']
religion_state_year['State'] =religion_state_year['State'].replace(state_abbrev_to_name)

years_to_drop = list(range(2000, 2014))

for years in years_to_drop:
    religion_state_year = religion_state_year[religion_state_year['Year'] != years]

state_counts = religion_state_year['State'].value_counts()

religion_state_year.to_csv('data/religion_data.csv', index=False)

## Eliza Preprocessing -- to 450

### Impute: 

### Clean:

## Abortion Preprocessing

The abortion data starts off as a table where each column is a type of abortion restriction and each row is for a law passed in a certain state. I start off by reshaping the data such that there is a column denoting the latest number of weeks at which abortion allowed (by the strictest law) in a given state and also added an indicator column denoting if there are no laws at all. Then, for the missing years (when no laws were enacted), I copied over the value from the previous year

### Reshape and Impute: 

In [16]:
# read data
abortion_data = pd.read_csv('data/abortion_data.csv')

# swap column names to be the number of weeks until abortion is banned
columns = ["State","Effective Date","Valid Through Date","Bans_gest_4 weeks postfertilization (6 weeks LMP) " ,"Bans_gest_6 weeks postfertilization (8 weeks LMP) " ,"Bans_gest_8 weeks postfertilization (10 weeks LMP)","Bans_gest_10 weeks postfertilization (12 weeks LMP) " ,"Bans_gest_12 weeks postfertilization (14 weeks LMP)","Bans_gest_13 weeks postfertilization (15 weeks LMP)","Bans_gest_16 weeks postfertilization (18 weeks LMP) ","Bans_gest_18 weeks postfertilization (20 weeks LMP)","Bans_gest_19 weeks postfertilization (21 weeks LMP)","Bans_gest20 weeks postfertilization (22 weeks LMP)","Bans_gest_21 weeks postfertilization (23 weeks LMP) " ,"Bans_gest_22 weeks postfertilization (24 weeks LMP)","Bans_gest_24 weeks postfertilization (26 weeks LMP)","Bans_gestViability","Bans_gest_Fetus is capable of feeling pain","Bans_gest_3rd trimester"]
abortion_data = abortion_data[columns]
new_names = {
    "State": "State",
    "Effective Date": "start",
    "Valid Through Date": "end",
    "Bans_gest_4 weeks postfertilization (6 weeks LMP) ": "4",
    "Bans_gest_6 weeks postfertilization (8 weeks LMP) ": "6",
    "Bans_gest_8 weeks postfertilization (10 weeks LMP)": "8",
    "Bans_gest_10 weeks postfertilization (12 weeks LMP) ": "10",
    "Bans_gest_12 weeks postfertilization (14 weeks LMP)": "12",
    "Bans_gest_13 weeks postfertilization (15 weeks LMP)": "13",
    "Bans_gest_16 weeks postfertilization (18 weeks LMP) ": "16",
    "Bans_gest_18 weeks postfertilization (20 weeks LMP)": "18",
    "Bans_gest_19 weeks postfertilization (21 weeks LMP)": "19",
    "Bans_gest20 weeks postfertilization (22 weeks LMP)": "20",
    "Bans_gest_21 weeks postfertilization (23 weeks LMP) ": "21",
    "Bans_gest_22 weeks postfertilization (24 weeks LMP)": "22",
    "Bans_gest_24 weeks postfertilization (26 weeks LMP)": "24",
    "Bans_gestViability": "24", #chose via google
    "Bans_gest_Fetus is capable of feeling pain": "25", #chose via google
    "Bans_gest_3rd trimester": "28" #chose via google
}
abortion_data = abortion_data.rename(columns=new_names)

# create a column, latest_abortion, that has the value of the number of weeks allowed until abortion is banned
abortion_processed = abortion_data[['State', 'start', 'end']].copy()
abortion_processed['latest_abortion'] = abortion_data[abortion_data.columns].apply(
    lambda row: next((int(col) for col in abortion_data.columns  if str(row[col]) == "1"), 40), axis=1)

# add an indicator column if there is no abortion law
abortion_processed['no_abortion_law'] = 0
abortion_processed.loc[abortion_processed['latest_abortion'] == 40, 'no_abortion_law'] = 1

# if there are multiple laws in a year, choose the one with the longest duration
abortion_processed['start'] = pd.to_datetime(abortion_processed['start'])
abortion_processed['end'] = pd.to_datetime(abortion_processed['end'])
abortion_processed['Year'] = abortion_processed['start'].dt.year
abortion_processed['length'] = (abortion_processed['end'] - abortion_processed['start']).dt.days
abortion_processed = abortion_processed.loc[abortion_processed.groupby(['State', 'Year'])['length'].idxmax()]
abortion_processed = abortion_processed.drop(columns=['start', 'end', 'length'])

# remove dc
abortion_no_dc = abortion_processed[abortion_processed['State'] != "District of Columbia"]

# create a dataframe include missing years
years = range(2018, 2022+1)
all_combinations = pd.MultiIndex.from_product(
    [abortion_no_dc['State'].unique(), years],
    names=['State', 'Year']
)

#impute with most recent law
abortion_state_year = abortion_no_dc.set_index(['State', 'Year']).reindex(all_combinations).reset_index()
abortion_state_year['latest_abortion'] = abortion_state_year.groupby('State')['latest_abortion'].ffill()
abortion_state_year['no_abortion_law'] = abortion_state_year.groupby('State')['no_abortion_law'].ffill()

## Now we combine our different datasets along State and Years

Note since we only have abortion data from 2018-2022, we made two datasetes, one including 2014-2017 and one without it

In [21]:
combined_2018_to_2022 = pd.merge(abortion_state_year, religion_state_year, on=['State', 'Year'], how='inner')
combined_2018_to_2022.head()

,State,Year,latest_abortion,no_abortion_law,Adherence Rate per 1000
0,Alabama,2018,20.0,0.0,634.654
1,Alabama,2019,20.0,0.0,635.332
2,Alabama,2020,20.0,0.0,636.010
3,Alabama,2021,20.0,0.0,636.688
4,Alabama,2022,20.0,0.0,637.366


In [22]:
combined_2014_to_2022 = pd.merge(abortion_state_year, religion_state_year, on=['State', 'Year'], how='outer')
combined_2014_to_2022.head()

,State,Year,latest_abortion,no_abortion_law,Adherence Rate per 1000
0,Alabama,2014,NaN,NaN,631.942
1,Alabama,2015,NaN,NaN,632.620
2,Alabama,2016,NaN,NaN,633.298
3,Alabama,2017,NaN,NaN,633.976
4,Alabama,2018,20.0,0.0,634.654


### One way to preliminarily understand why fertility rate is decreasing is to look at where it changes over time, and think about what factors dominate those areas.... aka do PCA

In [ ]:
pivoted_fertility = fertility_rate.pivot(index='STATE', columns='YEAR', values='FERTILITY RATE')
X_std =  StandardScaler().fit_transform(pivoted_fertility.fillna(0)) 
pca = PCA(n_components=1) 
principal_components = pca.fit_transform(X_std)

states = pivoted_fertility.index
pca_states = pd.DataFrame(data=principal_components, columns=['PC1'], index=states).reset_index()
pca_states.columns = ['state', 'PC1']

print("this accounts for", np.round(100*pca.explained_variance_ratio_[0], 2), "% variance")

this accounts for 90.25 % variance


In [ ]:
fig = px.choropleth(
    pca_states,
    locations='state',
    locationmode="USA-states",
    color='PC1',
    scope="usa",  
    range_color=[-5,5],
    color_continuous_scale=['purple', 'white', 'green'], 
    labels={'PC1': 'Principal Component Value'}
)

fig.update_layout(title_text="What states have the highest variation in fertility rates?")
fig.show()


We see that the first principle componetent represents 90% of varition over state and time. Thus, most of the change (net decrease) in fertility rate is seen when the coasts (purples) move strongly in one direction (presumably decrease given the net decrease) and the center of the US moves slightly in the other. We can use this information to understand what variables would be good predictors, aka the ones that match the coloring above. For example, a wealth distribution or political map would see similar differences with one color on the coast and another in the center. Thus we know that looking at household income and political affiliation are going to be really good predictor variables!!